In [ ]:
!pip install -U pip
!pip install -U dill
!pip install -U nltk==3.4

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.2.4)
Requirement already up-to-date: dill in /usr/local/lib/python3.6/dist-packages (0.3.2)
Requirement already up-to-date: nltk==3.4 in /usr/local/lib/python3.6/dist-packages (3.4)


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.util import pad_sequence
from nltk.util import bigrams, trigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk import word_tokenize, sent_tokenize 

In [ ]:
import os
import requests
import io
import re
import glob
import pandas as pd


In [ ]:
text = ''
for file_name in glob.glob('/content/drive/My Drive/TESI/paisa/'+'*.txt'):

  with io.open(file_name) as fin:
        
    text += fin.read()


In [ ]:
file_diz = pd.read_csv('/content/drive/My Drive/TESI/DIZIONARIO.csv', sep=';')

In [ ]:
dizionario = list(file_diz['TOKEN'])

In [ ]:
dizionario[:10]

['AICA',
 'AIMA',
 'AIR',
 'AIX',
 'AN',
 'AT',
 'AUDITEL',
 'AUSTRALIA',
 'Aardman',
 'Aaren']

In [ ]:
corpus = re.sub(r'<.*?>', '' , text)

In [ ]:
corpus = re.sub(r'\t', '' , corpus)

In [ ]:
sentences_split = list(map(str.lower, sent_tokenize(corpus, language='italian')))

In [ ]:
tokenized_text = [list(map(str.lower, word_tokenize(sent, language='italian'))) for sent in sentences_split]

In [ ]:
len(tokenized_text)

664199

In [ ]:
from nltk.lm.preprocessing import padded_everygram_pipeline

In [ ]:
diz, diz_pad = padded_everygram_pipeline(1,dizionario)

In [ ]:
n = 3
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)

In [ ]:
from nltk.lm import MLE

In [ ]:
model = MLE(n)

In [ ]:
model.fit(train_data, padded_sents)
print(model.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 658281 items>


In [ ]:
len(model.vocab)

658281

In [ ]:
model.score('mamma')

4.907639089427882e-05

In [ ]:
test_sentences = ""
for file_name in glob.glob('/content/drive/My Drive/TESI/paisa_02.txt'):

  with io.open(file_name) as fin:
        
    test_sentences = fin.read()

In [ ]:
sentences_split = list(map(str.lower, sent_tokenize(test_sentences, language='italian')))

In [ ]:
tokenized_ts = [list(map(str.lower, word_tokenize(sent, language='italian'))) for sent in sentences_split if sent!=[]]

In [ ]:
tokenized_ts = [tok for tok in tokenized_ts if tok!=[]]

In [ ]:
test_data, _ = padded_everygram_pipeline(3, tokenized_ts)

In [ ]:
import pickle

In [ ]:
pickle_out = open("/content/drive/My Drive/TESI/langmod.pickle","wb")
pickle.dump(model, pickle_out)
pickle_out.close()

**ESEMPI**

In [ ]:
testo = 'Carlo sa che Maria non lama più'

In [ ]:
tok_test = word_tokenize(testo.lower())

In [ ]:
trig = list(trigrams(tok_test))

In [ ]:
for t in tok_test:

  print(model.vocab.lookup(t))

carlo
sa
che
maria
non
lama
più


In [ ]:
for t in trig:

  print(t, model.score(t[2],[t[1]]))

('carlo', 'sa', 'che') 0.1658960606621047
('sa', 'che', 'maria') 6.068938961123214e-05
('che', 'maria', 'non') 0.01171875
('maria', 'non', 'lama') 3.94915073513441e-06
('non', 'lama', 'più') 0.0
